In [50]:
import pandas as pd
import numpy as np
import random 
from sklearn.model_selection import train_test_split

In [51]:
data = pd.read_csv("../../datasetProcessing/datasets/final.csv")

In [52]:
unique_classes = data['label'].unique()

# Randomly select two different classes
random_classes = random.sample(list(unique_classes), 2)

extracted_classes = data[data['label'].isin(random_classes)]

label_mapping = {random_classes[0]: 1, random_classes[1]: 0}
extracted_classes['label'] = extracted_classes['label'].map(label_mapping)

data = extracted_classes
print(data["label"].value_counts())

label
0    9634
1    9634
Name: count, dtype: int64


C:\Users\erick\AppData\Local\Temp\ipykernel_2068\4151952122.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_classes['label'] = extracted_classes['label'].map(label_mapping)


In [70]:
parts = 1
# Split features and labels
X = data.iloc[:int(data.shape[0]/parts), :-1]  # Features (all columns except the last one)
y = data.iloc[:int(data.shape[0]/parts), -1]

X_tr, X_te, y_tr, y_te = map(np.array, train_test_split(X, y, test_size=0.1, random_state=42))
print(X_tr.shape, y_tr.shape)
print(X_te.shape, y_te.shape)


(17341, 116) (17341,)
(1927, 116) (1927,)


In [71]:
import numpy as np 
from numpy import log,dot,exp,shape

def standardize(X_tr):
    for i in range(shape(X_tr)[1]):
        X_tr[:,i] = (X_tr[:,i] - np.mean(X_tr[:,i]))/np.std(X_tr[:,i])
def F1_score(y,y_hat):
    tp,tn,fp,fn = 0,0,0,0
    for i in range(len(y)):
        if y[i] == 1 and y_hat[i] == 1:
            tp += 1
        elif y[i] == 1 and y_hat[i] == 0:
            fn += 1
        elif y[i] == 0 and y_hat[i] == 1:
            fp += 1
        elif y[i] == 0 and y_hat[i] == 0:
            tn += 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2*precision*recall/(precision+recall)
    return f1_score
class LogisticRegression:
    def sigmoid(self,z):
        sig = 1/(1+exp(-z))
        return sig
    def initialize(self,X):
        weights = np.zeros((shape(X)[1]+1,1))
        X = np.c_[np.ones((shape(X)[0],1)),X]
        return weights,X
    def fit(self,X,y,alpha=0.001,iter=400):
        weights,X = self.initialize(X)
        def cost(theta):
            z = dot(X,theta)
            cost0 = y.T.dot(log(self.sigmoid(z)))
            cost1 = (1-y).T.dot(log(1-self.sigmoid(z)))
            cost = -((cost1 + cost0))/len(y)
            return cost
        cost_list = np.zeros(iter,)
        for i in range(iter):
            weights = weights - alpha*dot(X.T,self.sigmoid(dot(X,weights))-np.reshape(y,(len(y),1)))
            cost_list[i] = cost(weights)
        self.weights = weights
        return cost_list
    def predict(self,X):
        z = dot(self.initialize(X)[1],self.weights)
        lis = []
        for i in self.sigmoid(z):
            if i>0.5:
                lis.append(1)
            else:
                lis.append(0)
        return lis

In [72]:
obj1 = LogisticRegression()
model= obj1.fit(X_tr,y_tr)
y_pred = obj1.predict(X_te)
y_train = obj1.predict(X_tr)
#Let's see the f1-score for training and testing data
f1_score_tr = F1_score(y_tr,y_train)
f1_score_te = F1_score(y_te,y_pred)
print(f1_score_tr)
print(f1_score_te)

C:\Users\erick\AppData\Local\Temp\ipykernel_2068\117323648.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cost_list[i] = cost(weights)


1.0
1.0


In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
model = LogisticRegression().fit(X_tr,y_tr)
y_pred = model.predict(X_te)
print(f1_score(y_te,y_pred))

1.0
